<a href="https://colab.research.google.com/github/hlnynhi/PCAP/blob/main/segment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [20]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
df = pd.read_csv("https://raw.githubusercontent.com/hlnynhi/PCAP/main/Traffic-data/segment_status.csv", index_col="_id", parse_dates=["updated_at"])

print(df.head())
df.shape


                          updated_at  segment_id  velocity
_id                                                       
0   2020-07-03 14:55:31.869000+00:00       24845        20
1   2020-07-03 15:02:56.048000+00:00       33923        10
2   2020-07-04 08:15:52.696000+00:00       33824         5
3   2020-07-04 08:15:59.903000+00:00       33824         5
4   2020-07-04 08:16:08.201000+00:00       33824         5


(90938, 3)

In [17]:
segment_status_sort=segment_status.sort_values(by='velocity')
segment_status_sort

,_id,updated_at,segment_id,velocity
32416,32416,2020-08-03T00:20:31.797Z,29164,0
12828,12828,2020-08-03T00:05:30.888Z,83584,0
8119,8119,2020-08-03T00:02:28.071Z,6536,0
7466,7466,2020-08-02T23:59:32.926Z,62128,0
33760,33760,2020-08-03T00:23:27.901Z,5562,0
...,...,...,...,...
50856,50856,2020-12-02T15:33:41.160Z,14409,199
42002,42002,2020-11-26T01:00:22.535Z,63453,199
52438,52438,2020-12-04T02:19:19.741Z,68714,200
49320,49320,2020-12-02T02:21:46.673Z,10099,200


In [22]:
segReport_df = pd.read_csv("https://raw.githubusercontent.com/hlnynhi/PCAP/main/Traffic-data/segment_status.csv", index_col="_id", 
                            parse_dates=["updated_at"])
segment_df = pd.read_csv("https://raw.githubusercontent.com/hlnynhi/PCAP/main/Traffic-data/segments.csv", index_col="_id",
                         parse_dates=["created_at", "updated_at"])

In [23]:
from math import ceil

def transform_LOS(segment_id, velocity):
    max_velocity = segment_df.loc[segment_id, "max_velocity"]
    if max_velocity is None:
        max_velocity = 50
    
    # Transform to label
    labels = ["A", "B", "C", "D", "E", "F"]
    threshold = 35
    if max_velocity >= 70:
        threshold = 45
    elif max_velocity >= 60:
        threshold = 40

    t = max(threshold - velocity, 0)
    return labels[min(ceil(t / 5), 5)]

def transform_report(row):
    """
    @Params:
        dt: Timestamp object of Pandas
    @Return:
        dict: {"date", "period_{hour}_{00|30}"}
    """
    LOS = transform_LOS(row["segment_id"], row["velocity"])
    dt = row["updated_at"]
    intervals = list(range(24))
    h = dt.hour
    m = "00" if dt.minute < 30 else "30"
    p_name = f"period_{h}_{m}"
    return dt.date(), dt.weekday(), p_name, LOS

In [24]:
dates = []
weekdays = []
p_names = []
LOSes = []

for _, row in segReport_df.iterrows():
    date, weekday, p_name, LOS = transform_report(row)
    dates.append(date)
    weekdays.append(weekday)
    p_names.append(p_name)
    LOSes.append(LOS)

segReport_df["date"] = dates
segReport_df["weekday"] = weekdays
segReport_df["period"] = p_names
segReport_df["LOS"] = LOSes


In [25]:
def major_voting(labels):
    unique_labels = set(labels)
    count_labels = [labels.count(label) for label in unique_labels]

    sorted_labels = sorted(zip(unique_labels, count_labels), key=lambda x: x[1])
    if len(sorted_labels) > 1 and sorted_labels[0][1] == sorted_labels[1][1]:
        print("Oh no, many majors?")
    return sorted_labels[0][0]

def mean_voting(labels):
    l = ["A", "B", "C", "D", "E", "F"]
    values = {"A":0, "B":1, "C":2, "D":3, "E":4, "F":5}
    mean = sum(values[label] for label in labels) / len(labels)
    return l[min(round(mean), 5)]

In [26]:
compress_LOS = segReport_df.groupby(by=["segment_id", "date", "weekday", "period"])["LOS"].apply(list)
compress_LOS = pd.DataFrame(compress_LOS).reset_index()
compress_LOS["LOS"] = compress_LOS["LOS"].apply(mean_voting)
compress_LOS

,segment_id,date,weekday,period,LOS
0,26,2021-04-16,4,period_0_30,A
1,33,2020-08-02,6,period_23_30,C
2,33,2020-08-03,0,period_0_00,D
3,67,2021-03-09,1,period_9_30,B
4,67,2021-03-23,1,period_9_30,B
...,...,...,...,...,...
33436,84529,2021-01-05,1,period_8_00,F
33437,84533,2021-01-09,5,period_13_30,F
33438,84533,2021-04-22,3,period_5_00,F
33439,84534,2021-01-09,5,period_13_30,D
